Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Using Customized Modules to Create an Azure Machine Learning Pipeline

In this notebook, we will demonstrate how to use customized modules to create an Azure Machine Learning Pipeline which is associated with the [fastText](https://fasttext.cc/) algorithm. Customized modules are created with the extension azure-cli-ml of Azure CLI. 

If you don't have the input dataset, you could prepare your it with the instructions in ```prepare_data.ipynb```. 

The outline of this notebook is as follows:

- Upload the dataset onto a blob container and register it to the workspace.
- Register an anonymous module from yaml file to workspace.
- Construct the pipeline.
- Visualize and run the pipeline.

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [1]:
from azureml.core import Dataset, Datastore, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.data.datapath import DataPath
from azureml.pipeline.wrapper import Module, dsl

### Connect to workspace
Create a workspace object from the existing workspace. Workspace.from_config() reads the file config.json and loads the details into an object named workspace.

In [2]:
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute-gpu', 'aml-compute', 'my-compute'])


### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes. If the AmlCompute with that name is already in your workspace the code will skip the creation process.**


In [3]:
# aml_compute_name = 'aml-compute'
aml_compute_name = 'aml-compute-gpu' #GPU does not work
if aml_compute_name in workspace.compute_targets:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
else:
    print("Creating new compute target: {}".format(aml_compute_name))
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True)

Found existing compute target: aml-compute


### Upload the dataset onto a blob container and register it to the workspace.

In [4]:
dataset_name = "THUCNews"
# if the workspace don't contain the dataset, then register it
if not dataset_name in workspace.datasets:
    # upload files in 'data' to a blob container.
    # our files are in the directory of 'path_on_datastore' in the blob container.
    path_on_datastore = 'my_dataset'
    datastore = Datastore.get(workspace=workspace, datastore_name='workspaceblobstore')
    datastore.upload(src_dir='data/data_for_pipeline', target_path=path_on_datastore, overwrite=True, show_progress=True)
    description = 'THUCNews dataset is generated by filtering and filtering historical data \
    of Sina News RSS subscription channel from 2005 to 2011'
    # get the DataPath object associated with the uploaded dataset.
    datastore_path = [DataPath(datastore=datastore, path_on_datastore=path_on_datastore)]
    data = Dataset.File.from_files(path=datastore_path)
    # register the dataset to your workspace.
    data.register(workspace=workspace, name=dataset_name, description=description, create_new_version=True)
dataset = workspace.datasets[dataset_name]

Uploading an estimated of 3 files
Uploading data/data_for_pipeline\data.txt
Uploading data/data_for_pipeline\label.txt
Uploading data/data_for_pipeline\word_to_index.json
Uploaded data/data_for_pipeline\label.txt, 1 files out of an estimated total of 3
Uploaded data/data_for_pipeline\word_to_index.json, 2 files out of an estimated total of 3
Uploaded data/data_for_pipeline\data.txt, 3 files out of an estimated total of 3
Uploaded 3 files


### Register an anonymous module from yaml file to the workspace.
If you decorate your module function with ```@dsl.module```, azure-cli-ml could help to generate the ```*.spec.yaml``` file.

In [5]:
split_data_txt_module_func = Module.from_yaml(workspace, 'split_data_txt/split_data_txt.spec.yaml')
fasttext_train_module_func = Module.from_yaml(workspace, 'fasttext_train/fasttext_train.spec.yaml')
fasttext_evaluation_module_func = Module.from_yaml(workspace, 'fasttext_evaluation/fasttext_evaluation.spec.yaml')
compare_two_models_module_func = Module.from_yaml(workspace, 'compare_two_models/compare_two_models.spec.yaml')

### Construct the pipeline
our pipeline contains two sub pipelines. They represent two training processes of the fastText model with different parameters. 

In [6]:
# sub pipeline
@dsl.pipeline(name='sub_pipeline', description='A sub pipeline including processes of data processing/train/evaluation',
              default_compute_target=aml_compute_name)
def training_pipeline(epochs, batch_size, max_len):
    split_data_txt = split_data_txt_module_func(
        input_dir=dataset,
        training_data_ratio=0.7,
        validation_data_ratio=0.1
    )
    fasttext_train = fasttext_train_module_func(
        training_data_dir=split_data_txt.outputs.training_data_output,
        validation_data_dir=split_data_txt.outputs.validation_data_output,
        epochs=epochs,
        batch_size=batch_size,
        max_len=max_len,
        embed_dim=300,
        hidden_size=256,
        ngram_size=200000,
        learning_rate=0.001
    )

    fasttext_evaluation = fasttext_evaluation_module_func(
        trained_model_dir=fasttext_train.outputs.trained_model_dir,
        test_data_dir=split_data_txt.outputs.test_data_output
    )

    return {**fasttext_evaluation.outputs, **fasttext_train.outputs}

In [7]:
@dsl.pipeline(name='fasttext_pipeline',
              description='The pipeline that trains two fasttext models and output the better one',
              default_compute_target=aml_compute_name)
def fasttext_pipeline():
    train_and_evalute_model1 = training_pipeline(epochs=3, batch_size=64, max_len=32)
    train_and_evalute_model2 = training_pipeline(epochs=6, batch_size=64, max_len=32)
    compare = compare_two_models_module_func(
        first_trained_model=train_and_evalute_model1.outputs.trained_model_dir,
        first_trained_result=train_and_evalute_model1.outputs.model_testing_result,
        second_trained_model=train_and_evalute_model2.outputs.trained_model_dir,
        second_trained_result=train_and_evalute_model2.outputs.model_testing_result
    )
    return {**compare.outputs}


### Visualize and run the pipeline.

In [8]:
# get the pipeline
pipeline = fasttext_pipeline()
# save the pipeline if necessary
# pipeline.save(experiment_name=experiment_name)

In [9]:
# visualize the pipeline
pipeline.validate()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_eb359422-0603-4b28-976a-306deb9550ca_widget', env_json='{"subscription…

{'result': 'validation passed', 'errors': []}

In [10]:
# run the pipeline
experiment_name = 'fasttext_pipeline'
# regenerate_outputs indicates whether to force regeneration of all step outputs and disallow data reuse for this run
# if regenerate_outputs is False, this run may reuse results from previous runs and subsequent runs may reuse the results of this run
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=True)
# wait_for_completion() visualize the execution process of the pipeline
# you could also view this process on Azure Machine Learning Portal
# pipeline_run.wait_for_completion()

Submitted PipelineRun 42946bf0-b870-4af9-b230-7c257e9523d8
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_pipeline/runs/42946bf0-b870-4af9-b230-7c257e9523d8?wsid=/subscriptions/4f455bd0-f95a-4b7d-8d08-078611508e0b/resourcegroups/fundamental/workspaces/fundamental3
